## General Trees

A tree stores the data hieararchically. A parent node has from one to many other children, which also can have children. Formally, we define a tree T as a set of nodes storing elements such that the nodes
have a parent-child relationship that satisfies the following properties:

* If T is nonempty, it has a special node, called the root of T, that has no parent

* Each node v of T different frome the root has a unique parent node w; every node with parent w is a child of w

* Edges are connections between nodes

* Paths are ways how to go from a node to another on existing edges between the nodes



## Binary Trees

## Implementing Trees

## Tree Traversal Algorithms

## Exercises